# SimCLR on RPLAN Dataset
In this notebook, we try to project the RPLAN dataset into embedding space by SimCLR to evaluate SimCLR's performance. 

RPLAN dataset is a manually collected large-scale densely annotated dataset of floor plans from real residential buildings.[From dataset discription]

Here is an example in RPLAN dataset:

<img src="http://staff.ustc.edu.cn/~fuxm/projects/DeepLayout/DeepLayout.png" alt="examples" width="400"/>

[Link to SimCLR](https://arxiv.org/pdf/2002.05709.pdf)

[Link to RPLAN dataset](http://staff.ustc.edu.cn/~fuxm/projects/DeepLayout/index.html)

### The main work can be divided into two parts:

1. Change the binary images of RPLAN dataset to color images for better visualization
2. Use the color images to pre-train SimCLR model.
3. Use the pre-trained SimCLR model to project the RPLAN dataset into embedding space.
4. Use tensorboard to visualize the embedding space.

## Setup the repository

In [ ]:
!git clone https://github.com/spijkervet/SimCLR.git
%cd SimCLR
!mkdir -p logs && cd logs && wget https://github.com/Spijkervet/SimCLR/releases/download/1.2/checkpoint_100.tar && cd ../
!sh setup.sh || python3 -m pip install -r requirements.txt || exit 1
!pip install  pyyaml --upgrade
!pip install rplanpy

# Part1:
## Proprocessing the RPLAN dataset
In this part, we use rplanpy libary to read the RPLAN dataset and convert the binary images to color images. Here is two examples of this processing.

<img src="https://cdn.discordapp.com/attachments/884910103428476989/961703864225112074/unknown.png" alt="examples" width="400"/>

In [ ]:
import rplanpy
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2

In [ ]:
# import required module
import os
# assign directory
directory = 'rplan_dataset' # The place you store the dataset
number_of_images = 10000
# iterate over files in
# that directory
file_list = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
      file_list.append(f)

In [ ]:
cnt = 0
for filename in file_list[0:number_of_images]:
    cnt += 1
    if cnt%100==0:
      print('Processing image %d'%cnt)
    temp_1 = rplanpy.plot.floorplan_to_color(rplanpy.data.RplanData(filename))
    im = Image.fromarray(np.asarray(temp_1).astype(np.uint8))
    im.save("images/"+str(cnt)+".png")

# Part 2:
## SimCLR pre-training

In this part, we pre-train the SimCLR on the color images. The model we choose is ResNet18. And we just use the randomly initialized weights instead of pre-trained models.

### Import necessary libraries

In [ ]:
import os
import torch
import numpy as np  
import torchvision
import argparse
from torch.utils.tensorboard import SummaryWriter
apex = False
try:
    from apex import amp
    apex = True
except ImportError:
    print(
        "Install the apex package from https://www.github.com/nvidia/apex to use fp16 for training"
    )
from model import save_model, load_optimizer
from simclr import SimCLR
from simclr.modules import get_resnet, NT_Xent
from simclr.modules.transformations import TransformsSimCLR

Install the apex package from https://www.github.com/nvidia/apex to use fp16 for training


### Load arguments from `config/config.yaml`

In [ ]:
from pprint import pprint
import argparse
from utils import yaml_config_hook

parser = argparse.ArgumentParser(description="SimCLR")
config = yaml_config_hook("./config/config.yaml")
for k, v in config.items():
    parser.add_argument(f"--{k}", default=v, type=type(v))

args = parser.parse_args([])
args.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
### override any configuration parameters here, e.g. to adjust for use on GPUs on the Colab platform:
args.batch_size = 128
args.resnet = "resnet18"
pprint(vars(args))

{'batch_size': 128,
 'dataparallel': 0,
 'dataset': 'CIFAR10',
 'dataset_dir': './datasets',
 'device': device(type='cuda'),
 'epoch_num': 100,
 'epochs': 100,
 'gpus': 1,
 'image_size': 224,
 'logistic_batch_size': 256,
 'logistic_epochs': 500,
 'model_path': 'save',
 'nodes': 1,
 'nr': 0,
 'optimizer': 'Adam',
 'pretrain': True,
 'projection_dim': 64,
 'reload': False,
 'resnet': 'resnet18',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 8}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/

/content/drive/MyDrive/test_set_rplan


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
from torchvision import datasets, transforms
import torchvision
import matplotlib.pyplot as plt
import copy
import random
from PIL import Image
import PIL.ImageOps
import random

### Load dataset into train loader

In [ ]:
torch.manual_seed(args.seed)
np.random.seed(args.seed)
train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/test_set_rplan/",transform=TransformsSimCLR(size=args.image_size))
print(len(train_dataset))
if args.nodes > 1:
    train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset, num_replicas=args.world_size, rank=rank, shuffle=True
    )
else:
    train_sampler = None

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=(train_sampler is None),
    drop_last=True,
    num_workers=args.workers,
    sampler=train_sampler,
)

5250


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### Load the SimCLR model, optimizer and learning rate scheduler

In [ ]:
# initialize ResNet
encoder = get_resnet(args.resnet, pretrained=False)
n_features = encoder.fc.in_features  # get dimensions of fc layer

# initialize model
model = SimCLR(encoder, args.projection_dim, n_features)
#model.load_state_dict(torch.load("/content/drive/MyDrive/checkpoint_100.tar", map_location=args.device.type))

model = model.to(args.device)

# optimizer / loss
optimizer, scheduler = load_optimizer(args, model)

### Initialize the criterion (NT-Xent loss)

In [ ]:
criterion = NT_Xent(args.batch_size, args.temperature, world_size=1)

### Setup TensorBoard for logging experiments

In [ ]:
writer = SummaryWriter()

### Train function

In [ ]:
def train(args, train_loader, model, criterion, optimizer, writer):
    loss_epoch = 0
    for step, ((x_i, x_j), _) in enumerate(train_loader):
        optimizer.zero_grad()
        x_i = x_i.cuda(non_blocking=True)
        x_j = x_j.cuda(non_blocking=True)

        # positive pair, with encoding
        h_i, h_j, z_i, z_j = model(x_i, x_j)
        loss = criterion(z_i, z_j)
        loss.backward()

        optimizer.step()

        if step % 50 == 0:
            print(f"Step [{step}/{len(train_loader)}]\t Loss: {loss.item()}")

        writer.add_scalar("Loss/train_epoch", loss.item(), args.global_step)
        loss_epoch += loss.item()
        args.global_step += 1
    return loss_epoch


### Start training

In [ ]:
args.global_step = 0
args.current_epoch = 0
print(len(train_loader))
for epoch in range(args.start_epoch, args.epochs):
    lr = optimizer.param_groups[0]["lr"]
    loss_epoch = train(args, train_loader, model, criterion, optimizer, writer)

    if scheduler:
        scheduler.step()

    # save every 10 epochs
    '''
    if epoch % 10 == 0:
        save_model(args, model, optimizer)
    '''

    writer.add_scalar("Loss/train", loss_epoch / len(train_loader), epoch)
    writer.add_scalar("Misc/learning_rate", lr, epoch)
    print(
        f"Epoch [{epoch}/{args.epochs}]\t Loss: {loss_epoch / len(train_loader)}\t lr: {round(lr, 5)}"
    )
    args.current_epoch += 1

# end training


### Save the checkpoints

In [ ]:
args.model_path = "/content/drive/MyDrive/rplan"
save_model(args, model, optimizer)

# Part 3:
## Project the train set to the embedding space.

Here, we only project the train set to the embedding space to save some time. (Because some errors in google drive, we only load 5000+ images into the train set.)

In [ ]:
import torch
import torchvision
import numpy as np
import argparse

### Load the training set with no shuffle.

In [ ]:
train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/test_set_rplan/",transform=TransformsSimCLR(size=args.image_size).test_transform)
print(len(train_dataset))
if args.nodes > 1:
    train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset, num_replicas=args.world_size, rank=rank, shuffle=True
    )
else:
    train_sampler = None

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=None,
    drop_last=True,
    num_workers=args.workers,
    sampler=train_sampler,
)

5250


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
from pprint import pprint

args = parser.parse_args([])

args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Load dataset into train/test dataloaders

### Helper functions to map all input data $X$ to their latent representations $h$.

In [ ]:
def inference(loader, simclr_model, device):
    feature_vector = []
    labels_vector = []
    for step, (x, y) in enumerate(loader):
        x = x.to(device)

        # get encoding
        with torch.no_grad():
            h = simclr_model(x)

        h = h.detach()

        feature_vector.extend(h.cpu().detach().numpy())
        labels_vector.extend(y.numpy())

        if step % 20 == 0:
            print(f"Step [{step}/{len(loader)}]\t Computing features...")

    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector


def get_features(context_model, train_loader, device):
    train_X, train_y = inference(train_loader, context_model, device)
    #test_X, test_y = inference(test_loader, context_model, device)
    return train_X, train_y#test_X, test_y


In [ ]:
print("### Creating features from pre-trained context model ###")
(train_X, train_y) = get_features(
    encoder, train_loader, args.device
)

### Creating features from pre-trained context model ###


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Step [0/41]	 Computing features...
Step [20/41]	 Computing features...
Step [40/41]	 Computing features...
Features shape (5248, 512)


# Part 4:
## Data Visualization

In this part, we will visualize the embedding space of the train set. We refer to the blog [How to visualize image feature vectors](https://hanna-shares.medium.com/how-to-visualize-image-feature-vectors-1e309d45f28f). And we will use tensorboard visualize the embedding space.

In [ ]:
import pandas as pd
from PIL import Image
import csv

In [ ]:
# transform the tensor to PIL image
trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize([56, 56])                           
  ]
)
DatasetPIL = []
for (images,_) in train_loader:
  for image in images:
    DatasetPIL.append(trans(image))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### Use the sprite library to aggregate all images into 1 images
![](https://media.discordapp.net/attachments/884910103428476989/961711807645507684/unknown.png?width=1296&height=1302)

In [ ]:
image_number = 3000
images = DatasetPIL[:3000]

image_width, image_height = images[0].size
one_square_size = int(np.ceil(np.sqrt(len(images))))
master_width = (image_width * one_square_size) 
master_height = image_height * one_square_size
spriteimage = Image.new(
    mode='RGBA',
    size=(master_width, master_height),
    color=(0,0,0,0))  # fully transparent
for count, image in enumerate(images):
    div, mod = divmod(count,one_square_size)
    h_loc = image_width*div
    w_loc = image_width*mod    
    spriteimage.paste(image,(w_loc,h_loc))
spriteimage.convert("RGB").save('vis/sprite56.jpg', transparency=0)

### Write all the embeddings into one csv file

In [ ]:
vecs = [vec for vec in train_X[:image_number]]
with open('vis/feature_vecs56.tsv', 'w+') as fw:
    csv_writer = csv.writer(fw, delimiter='\t')
    csv_writer.writerows(vecs)

# we will also create the config file for tensorboard to identify the tensor and image
`embeddings {
  tensor_path: "feature_vecs.tsv"
  metadata_path: "metadata.tsv"
  sprite {
    image_path: "sprite.jpg"
    single_image_dim: 50
    single_image_dim: 50
  }
}`
# then run tensorboard
`tensorboard --logdir ./vis`